In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas
from torch.utils.data import Dataset
import os

In [2]:
label_mapping = {
    0: 0,  # T-shirt/top -> Upper
    1: 1,  # Trouser -> Lower
    2: 1,  # Pullover -> Lower
    3: 0,  # Dress -> Upper
    4: 0,  # Coat -> Upper
    5: 2,  # Sandal -> Feet
    6: 0,  # Shirt -> Upper
    7: 2,  # Sneaker -> Feet
    8: 3,  # Bag -> Bag
    9: 2,  # Ankle boot -> Feet
}

Word_mapping = {
    0: "Upper",  # 4000
    1: "Lower",  # 2000
    3: "Bag",  # 1000
    2: "Feet"  # 3000
}

Num_mapping = {
    0: 4000,
    1: 2000,
    3: 1000,
    2: 3000
}

In [3]:

def CalSpace(msg):
    return 14 - len(msg)


def MakeSpace(msg):
    req = CalSpace(msg)
    res = ""

    for _ in range(req):
        res = res + " "

    return res


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:

class NewTrainDataset(Dataset):
    def __init__(self, train_dataframe_feature, train_dataframe_label):
        self.dataf = train_dataframe_feature
        self.datal = train_dataframe_label

    def __getitem__(self, index):
        return self.dataf[index], label_mapping[self.datal[index]]

    def __len__(self):
        return len(self.datal)


class NewTestDataset(Dataset):
    def __init__(self, test_dataframe_feature, test_dataframe_label):
        self.dataf = test_dataframe_feature
        self.datal = test_dataframe_label

    def __getitem__(self, index):
        return self.dataf[index], label_mapping[self.datal[index]]

    def __len__(self):
        return len(self.datal)

In [6]:

class ConvNet(nn.Module):
    def __init__(self, hidden1, hidden2, num_out):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3)
        self.mxpl1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=4)
        self.mxpl2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.linear1 = nn.Linear(
            in_features=250, out_features=hidden1, bias=True)
        self.linear2 = nn.Linear(
            in_features=hidden1, out_features=hidden2, bias=True)
        self.linear3 = nn.Linear(
            in_features=hidden2, out_features=num_out, bias=False)

    def forward(self, data):
        out = nn.Tanh()(self.mxpl1(self.conv1(data)))
        out = nn.Tanh()(self.mxpl2(self.conv2(out)))

        out = out.view(-1, 250)

        out = nn.ReLU()(self.linear1(out))
        out = nn.ReLU()(self.linear2(out))
        out = self.linear3(out)  # RelU not Req

        return out

# Layer conv => maxpool > activation(TanH) > conv => maxpool => activation(TanH) => Linear => activation (RelU)
# => Linear => activation (RelU) => Linear => activation (Not req , Cross Entropy takes care , SoftMax)

In [7]:
train_data = datasets.FashionMNIST(
    root='data',
    download=True,
    train=True,
    transform=ToTensor()
)

print("Train Dataset Loaded")

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=False,
    transform=ToTensor()
)

print("Test Datset Loaded")

100%|██████████| 26421880/26421880 [00:01<00:00, 18305843.30it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 337419.65it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6038510.12it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4694993.91it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Train Dataset Loaded
Test Datset Loaded


In [8]:
train_dataframe_feature = pandas.DataFrame(
    data=(torch.flatten(train_data.data, start_dim=1)).numpy())
print("Train Dataset Feature Shape {0}".format(
    train_dataframe_feature.shape))

test_dataframe_feature = pandas.DataFrame(
    data=torch.flatten(input=test_data.data, start_dim=1).numpy())
print("Test Dataframe Feature Shape {0}".format(
    test_dataframe_feature.shape))

Train Dataset Feature Shape (60000, 784)
Test Dataframe Feature Shape (10000, 784)


In [9]:
train_dataframe_label = pandas.DataFrame(data=train_data.targets.numpy())
print("Train Dataframe Label Shape {0}".format(
    train_dataframe_label.shape))

test_dataframe_label = pandas.DataFrame(data=test_data.targets.numpy())
print("Test Dataframe Label Shape {0}".format(test_dataframe_label.shape))

Train Dataframe Label Shape (60000, 1)
Test Dataframe Label Shape (10000, 1)


In [10]:
print("Target Labels {0}".format(train_data.classes))

Target Labels ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


In [11]:
print("Creating new datasets")

new_train_dataset = NewTrainDataset(train_dataframe_feature=train_data.data.numpy(
), train_dataframe_label=train_data.targets.numpy())
new_test_dataset = NewTestDataset(test_dataframe_feature=test_data.data.numpy(
), test_dataframe_label=test_data.targets.numpy())

Creating new datasets


In [12]:

batch_size = 100
learning_rate = 1e-2
conv_input_size = 125
conv_hidden_size_1 = 256
conv_hidden_size_2 = 256
num_class_out = 4
epoches = 5

FILE_PATH = "./model/label10_model.pth"

if not os.path.exists("./model"):
    os.mkdir("./model")

In [13]:
def ViewData(data, prefix=""):
    x, y = next(iter(data))
    print("{1} Feature Shape {0}".format(x.shape, prefix))
    print("{1} Label Shape {0}".format(y.shape, prefix))


new_train_dataset_dataloader = DataLoader(
dataset=new_train_dataset, num_workers=2, shuffle=True, batch_size=batch_size)
new_test_dataset_dataloader = DataLoader(
dataset=new_test_dataset, num_workers=2, shuffle=False, batch_size=batch_size)

ViewData(new_train_dataset_dataloader, "NewTrain")
ViewData(new_test_dataset_dataloader, "NewTest")

NewTrain Feature Shape torch.Size([100, 28, 28])
NewTrain Label Shape torch.Size([100])
NewTest Feature Shape torch.Size([100, 28, 28])
NewTest Label Shape torch.Size([100])


In [14]:
print("Device used {0}".format(device.type))

Device used cpu


In [15]:
model = ConvNet(hidden1=conv_hidden_size_1,
                hidden2=conv_hidden_size_2, num_out=num_class_out)
model.to(device=device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [17]:
if os.path.exists(FILE_PATH):
    print("Loading Model")
    model.load_state_dict(torch.load(FILE_PATH))
else:
    print("Saving Model")
    losses = []

    model.train(mode=True)
    for epoch in range(epoches):
        temp =[]
        for i, (features, labels) in enumerate(new_train_dataset_dataloader):
            features = torch.as_tensor(
                data=features, dtype=torch.float32, device=device)
            labels = torch.as_tensor(
                data=labels, dtype=torch.long, device=device)
            features = features.unsqueeze(dim=1)

            pred_y = model(features)

            loss = loss_func(pred_y, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            temp.append(loss.item())

            if i % 100 == 0:
                print("Epoch [{0}/{1}] Iter [{2}/{3}] Loss [{4:.5f}]".format(
                    epoch+1, epoches, i+100, 60000/batch_size, loss.item()))
        losses.append(temp)

    model.train(mode=False)

    for index in range(len(losses)):
        lossesv = losses[index]
        print("\n\nEpoch           {0}".format(index+1))
        print("------------------------")

        avg = 0.0
        for value in lossesv:
            avg += value / batch_size
        
        print("Average loss      {0:.5f}".format(avg))


    torch.save(model.state_dict(), FILE_PATH)


Saving Model
Epoch [1/5] Iter [100/600.0] Loss [0.22570]
Epoch [1/5] Iter [200/600.0] Loss [0.16074]
Epoch [1/5] Iter [300/600.0] Loss [0.21079]
Epoch [1/5] Iter [400/600.0] Loss [0.05851]
Epoch [1/5] Iter [500/600.0] Loss [0.12005]
Epoch [1/5] Iter [600/600.0] Loss [0.14605]
Epoch [2/5] Iter [100/600.0] Loss [0.17093]
Epoch [2/5] Iter [200/600.0] Loss [0.14522]
Epoch [2/5] Iter [300/600.0] Loss [0.09858]
Epoch [2/5] Iter [400/600.0] Loss [0.08663]
Epoch [2/5] Iter [500/600.0] Loss [0.20153]
Epoch [2/5] Iter [600/600.0] Loss [0.12486]
Epoch [3/5] Iter [100/600.0] Loss [0.09861]
Epoch [3/5] Iter [200/600.0] Loss [0.14354]
Epoch [3/5] Iter [300/600.0] Loss [0.09341]
Epoch [3/5] Iter [400/600.0] Loss [0.13002]
Epoch [3/5] Iter [500/600.0] Loss [0.15760]
Epoch [3/5] Iter [600/600.0] Loss [0.11285]
Epoch [4/5] Iter [100/600.0] Loss [0.15624]
Epoch [4/5] Iter [200/600.0] Loss [0.09671]
Epoch [4/5] Iter [300/600.0] Loss [0.22244]
Epoch [4/5] Iter [400/600.0] Loss [0.20442]
Epoch [4/5] Iter [5

In [18]:
print("\n\nTesting Model")
with torch.no_grad():
    model.eval()
    test_tot_loss = 0.0
    correct = 0.0
    correctFE = [0.0 for i in range(4)]
    boolCrt = []
    for i, (features, labels) in enumerate(new_test_dataset_dataloader):
        features = torch.as_tensor(
            features, dtype=torch.float32, device=device)
        labels = torch.as_tensor(
            data=labels, dtype=torch.long, device=device)

        features = features.squeeze(dim=0)
        features = features.unsqueeze(dim=1)

        pred_y = model(features)
        loss = loss_func(pred_y, labels)
        test_tot_loss += (loss.item() / batch_size)

        preVal, preValIndexes = torch.max(pred_y, dim=1)

        for index in range(len(preVal)):
            if torch.eq(preValIndexes[index], labels[index]):
                boolCrt.append(preValIndexes[index])

    for vi in boolCrt:
        correctFE[vi] += 1
        correct += 1

    print("\nOverall Accuracy   [{0:.5f}]".format(correct/10000))
    print("Name          Accuracy")
    print("--------------------------")
    for index, value in enumerate(correctFE):
        print("{0}{1}[{2:.5f}]".format(Word_mapping[index], MakeSpace(Word_mapping[index]), value/Num_mapping
                                        [index]))



Testing Model

Overall Accuracy   [0.92980]
Name          Accuracy
--------------------------
Upper         [0.96925]
Lower         [0.74600]
Feet          [0.99600]
Bag           [0.94100]
